드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

내 드라이브로 이동 (또는 project_space 디렉토리로 이동)

In [ ]:
%cd /content/drive/MyDrive/

git clone 밑 requirements 설치

In [ ]:
!git clone https://github.com/cszn/KAIR.git

In [ ]:
%cd KAIR
!pwd

In [ ]:
!pip install -r requirement.txt

테스트셋 폴더 생성

In [ ]:
!mkdir -p testsets/Set5/HR testsets/Set5/LR_bicubic/X4

[KAIR]  

  |--[trainsets]  

  |--|--[trainH] // put your train images here.  

  |  

  |--[testsets]  

  |--|--[Set5]  

  |--|--|--[HR]  put your test image heres. (default: 256*256)

  |--|--|--[LR_bicubic]




Make LR test image

In [ ]:
from PIL import Image
import os

# 디렉토리 설정
hr_dir = "testsets/Set5/HR"  # HR 이미지를 저장할 디렉토리
lr_dir = "testsets/Set5/LR_bicubic/X4"  # 생성된 LR 이미지를 저장할 디렉토리
os.makedirs(lr_dir, exist_ok=True)

# HR 이미지를 읽고 X4 다운스케일링하여 LR 생성
for img_name in os.listdir(hr_dir):
    hr_path = os.path.join(hr_dir, img_name)
    lr_path = os.path.join(lr_dir, img_name)

    # 파일만 처리하고 디렉토리는 무시
    if os.path.isfile(hr_path):  # 디렉토리 무시 조건 추가
        with Image.open(hr_path) as img:
            # X4 다운스케일링
            lr_img = img.resize((img.width // 4, img.height // 4), Image.BICUBIC)
            lr_img.save(lr_path)

print(f"LR images saved to {lr_dir}")

Before training, modify the parameters and definitions in train_swinir_sr_realworld_x4_psnr.json, main_train_psnr.py, model_base.py, model_plain.py, and utils_blindsr.py according to your requirements. after running train code, superresolution directory will created. put your pretrained .pth file in superresolution/models.

In [ ]:
!python main_train_psnr.py --opt options/swinir/train_swinir_sr_realworld_x4_psnr.json

Extract Average PSNR from train.log after training.

In [ ]:
import re

def extract_log_info(log_file_path, output_file_path):
    with open(log_file_path, 'r', encoding='utf-8') as log_file:
        lines = log_file.readlines()

    pretrained_netG_pattern = r'pretrained_netG:\s*(.*?)\n'
    epoch_psnr_pattern = r'<epoch:\s*(\d+),\s*iter:\s*(\d+(?:,\d+)*),\s*Average PSNR\s*:\s*([\d\.]+)dB'

    extracted_info = []

    for line in lines:
        pretrained_match = re.search(pretrained_netG_pattern, line)
        epoch_psnr_match = re.search(epoch_psnr_pattern, line)

        if pretrained_match:
            extracted_info.append(f'pretrained_netG: {pretrained_match.group(1)}')
        elif epoch_psnr_match:
            epoch = epoch_psnr_match.group(1)
            iteration = epoch_psnr_match.group(2)
            psnr = epoch_psnr_match.group(3)
            extracted_info.append(f'<epoch: {epoch}, iter: {iteration}, Average PSNR : {psnr}dB')

    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for info in extracted_info:
            output_file.write(info + '\n')

# 샘플 로그 파일 경로와 출력 파일 경로 설정
log_file_path = "path_of_your_trainlog_file_directory"
output_file_path = "path_of_your_extracted_output_file_directory"

# 함수 실행하여 텍스트 추출 및 저장
extract_log_info(log_file_path, output_file_path)


Draw graph

In [ ]:
import re
import matplotlib.pyplot as plt

def extract_log_info_with_lr(log_file_path):
    with open(log_file_path, 'r', encoding='utf-8') as log_file:
        lines = log_file.readlines()

    epoch_psnr_pattern = r'<epoch: +?(\d+), iter: +(\d{1,3}(?:,\d{3})?), Average PSNR : ([\d\.]+)dB'

    iterations = []
    psnr_values = []

    for line in lines:
        epoch_psnr_match = re.search(epoch_psnr_pattern, line)
        if epoch_psnr_match:
            iteration = epoch_psnr_match.group(2)
            psnr = epoch_psnr_match.group(3)
            iterations.append(int(iteration.replace(',', '')))
            psnr_values.append(float(psnr))

    # Plotting the graph for iterations vs Average PSNR
    plt.figure(figsize=(10, 6))
    plt.plot(iterations, psnr_values, label='Average PSNR', color='b', marker='o')

    # Mark the learning rate change at iteration 12500
    #lr_change_iteration = 12500
    #plt.axvline(x=lr_change_iteration, color='r', linestyle='--', label='lr changed (1e-4 → 5e-5)')
    #plt.text(lr_change_iteration, min(psnr_values), 'lr: 1e-4 → 5e-5', color='red', fontsize=10, rotation=90, verticalalignment='bottom')

    # Add labels, title, and legend
    plt.xlabel('Iteration')
    plt.ylabel('Average PSNR (dB)')
    plt.title('Average PSNR Over Iterations with Learning Rate Change')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# 샘플 로그 파일 경로 설정
log_file_path = "path_of_your_extracted_output_file_directory"

# 함수 실행하여 그래프 그리기
extract_log_info_with_lr(log_file_path)
